In [30]:
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install sqlalchemy




  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/68/89/247a6a5bfcc5f2f27b08ce7a1fbc2ee9e54fa09ccc9fa48e8c0e08e78cbd/SQLAlchemy-2.0.21-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/192.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/192.5 kB ? eta -:--:--
     ------------- ----------------------- 71.7/192.5 kB 975.2 kB/s eta 0:00:01
     -------------------------------------- 192.5/192.5 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 25.1 MB/s eta 0:00:01
   ---------------------- ------

In [41]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine

host=os.getenv("DB_HOST")
database=os.getenv("DB_NAME")
user=os.getenv("DB_USERNAME")
password=os.getenv("DB_PASSWORD")


conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()
# Create the SQLAlchemy engine using the MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

table_name = "stg_season_schedule"  # Name of the SQL table
#df.to_sql(table_name, engine, if_exists='replace', index=False, method='multi')

#mycursor.execute("CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")
sql_query = f"SELECT * FROM {table_name}"

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
#conn.close()

In [38]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #print(f'game_id: {gameID} team_id: {teamID} season:{seasonYear}')
        gameStats = cumestatsteam.CumeStatsTeam(game_ids='00'+str(gameID),league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        #print(gameStats)
        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    #print(data)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [52]:
sql_query = """ SELECT 
                    sss.* 
                FROM 
                    stg_season_schedule sss
                LEFT JOIN 
                    stg_game_stats sgs1 on sss.game_id = sgs1.game_id and sss.home_team_id = sgs1.team_id
                LEFT JOIN 
                    stg_game_stats sgs2 on sss.game_id = sgs2.game_id and sss.away_team_id = sgs2.team_id
                WHERE 
                    sgs1.team_id is null
                      """

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
df

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID
0,05/13/2021 Magic at Hawks,22001049,2020-21,2021-05-13,Hawks,1610612737,Magic,1610612753
1,05/12/2021 Wizards at Hawks,22001042,2020-21,2021-05-12,Hawks,1610612737,Wizards,1610612764
2,05/10/2021 Wizards at Hawks,22001026,2020-21,2021-05-10,Hawks,1610612737,Wizards,1610612764
3,05/06/2021 Hawks at Pacers,22001000,2020-21,2021-05-06,Pacers,1610612754,Hawks,1610612737
4,05/05/2021 Suns at Hawks,22000992,2020-21,2021-05-05,Hawks,1610612737,Suns,1610612756
...,...,...,...,...,...,...,...,...
3534,10/25/2022 Pistons at Wizards,22200052,2022-23,2022-10-25,Wizards,1610612764,Pistons,1610612765
3535,03/09/2023 Hornets at Pistons,22200992,2022-23,2023-03-09,Pistons,1610612765,Hornets,1610612766
3536,02/27/2023 Pistons at Hornets,22200919,2022-23,2023-02-27,Hornets,1610612766,Pistons,1610612765
3537,02/03/2023 Hornets at Pistons,22200785,2022-23,2023-02-03,Pistons,1610612765,Hornets,1610612766


In [54]:
failed_attempts = 0
for _,row in df.iterrows():
    try:
        getSingleGameMetrics(row['GAME_ID'],row['HOME_TEAM_ID'],row['AWAY_TEAM_ID'],row['AWAY_TEAM_NICKNAME'],row['SEASON'],row['GAME_DATE']).to_sql('stg_game_stats', engine, if_exists='append', index=False, method='multi')
    except:
        print(f'failed attempt number {failed_attempts}')
        failed_attempts+= 1
        continue

HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


In [34]:
# insert into table the new record
# based on existing records determine records to pull 

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID
0,05/16/2021 Rockets at Hawks,22001066,2020-21,2021-05-16,Hawks,1610612737,Rockets,1610612745
1,05/13/2021 Magic at Hawks,22001049,2020-21,2021-05-13,Hawks,1610612737,Magic,1610612753
2,05/12/2021 Wizards at Hawks,22001042,2020-21,2021-05-12,Hawks,1610612737,Wizards,1610612764
3,05/10/2021 Wizards at Hawks,22001026,2020-21,2021-05-10,Hawks,1610612737,Wizards,1610612764
4,05/06/2021 Hawks at Pacers,22001000,2020-21,2021-05-06,Pacers,1610612754,Hawks,1610612737
...,...,...,...,...,...,...,...,...
3535,10/25/2022 Pistons at Wizards,22200052,2022-23,2022-10-25,Wizards,1610612764,Pistons,1610612765
3536,03/09/2023 Hornets at Pistons,22200992,2022-23,2023-03-09,Pistons,1610612765,Hornets,1610612766
3537,02/27/2023 Pistons at Hornets,22200919,2022-23,2023-02-27,Hornets,1610612766,Pistons,1610612765
3538,02/03/2023 Hornets at Pistons,22200785,2022-23,2023-02-03,Pistons,1610612765,Hornets,1610612766


In [22]:
from nba_api.stats.endpoints import playercareerstats
import requests

beg = 'http://'


headers = requests.utils.default_headers()

headers.update(
    {'Host': 'stats.nba.com', 
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0', 
    'Accept': 'application/json, text/plain, */*', 
    'Accept-Language': 'en-US,en;q=0.5', 
    'Accept-Encoding': 'gzip, deflate, br', 
    'x-nba-stats-origin': 'stats', 
    'x-nba-stats-token': 'true', 
    'Connection': 'keep-alive', 
    'Referer': 'https://stats.nba.com/', 
    'Pragma': 'no-cache', 
    'Cache-Control': 'no-cache'}
)
# Create the proxy URL with credentials
#proxy_url_with_credentials = f"{beg}{username}:{password}@{proxy_ip}:{proxy_port}"


career = playercareerstats.PlayerCareerStats(player_id='203999',proxy=proxy',headers=headers,timeout=600) 